# Basic RAG

Hello everyone! We'll discuss about Basic RAG here.

RAG (Retrieval Augmented Generation) is a technique in large language model app to connect model with outside database. 

The main idea behind RAG is to store a collection of documents in a vector store and use it to answer questions by finding the most relevant documents and combining their content.

Topics we'll cover are:

1. [Text and Document in Langchain](#1)
2. [Create Vector Store in Local Database](#2)
3. [Create Vector Store in Cloud Database](#3)
4. [Query Vector Store](#4)
5. [Retriever](#5)
6. [Chain with Retriever](#6)

Source:
https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 1 - Text and Document <div id="1"></div>

LangChain supports both text and document data in its RAG. A text-based RAG works with plain strings, while a document-based RAG works with Document objects.

In Langchain, `Document` is represent a unit of text and associated metadata. It has two attributes:

- `page_content`: a string representing the content;
- `metadata`: a dict containing arbitrary metadata.

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

Here we've generated six documents, containing metadata indicating three distinct "sources".

## 2- Create a Vector Store on Local <div id="2"></div>

As the name suggests, a vector store stores embedding vectors derived from documents or text. This is a common method for storing and searching through unstructured data.

By using embedding vectors, we obtain a semantic representation of each document, which allows us to calculate the relationships or similarities between a query and the documents.

Let's start with defining embedding function

In [3]:
from langchain_openai import OpenAIEmbeddings

embedding=OpenAIEmbeddings(model="text-embedding-3-small")

Now, we can create a Vector Store directly on our local computer using Chroma.

In [5]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding,
    persist_directory="db/doc-db" # must be specified, if not it will be crashed
)

We can also store plain text

In [6]:
texts = [
    "Dogs are great companions, known for their loyalty and friendliness.",
    "Cats are independent pets that often enjoy their own space.",
    "Goldfish are popular pets for beginners, requiring relatively simple care.",
    "Parrots are intelligent birds capable of mimicking human speech.",
    "Rabbits are social animals that need plenty of space to hop around.",
]

vector1 = Chroma.from_texts(
    texts,
    embedding=embedding,
    persist_directory="text-db" # must be specified, if not it will be crashed
)

## 3 - Vector Store in Cloud Server <div id="3"></div>

We can also store vector store in Cloud Server such as pinecone. You have to specify `PINECONE_API_KEY` before running block below and create database with index_name="db" in pinecone db

In [7]:
from langchain_pinecone import PineconeVectorStore 

pinecode = PineconeVectorStore.from_texts(
    texts, embedding, index_name="db"
)

d:\Document\Machine_Learning_Journey\Projects\Langchain\learn-langchain-from-scratch\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## 4 - Query Data from Vector Store <div id="4"></div>

Once we've instantiated a `VectorStore` that contains documents, we can query it. VectorStore includes methods for querying:

- Synchronously and asynchronously;
- By string query and by vector;
- With and without returning similarity scores;
- By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).

In [10]:
await vectorstore.similarity_search('cat', k=2)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [18]:
vectorstore.similarity_search_with_score("mammal", k=2)

[(Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.5499376971513037),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5651748849858715)]

In [15]:
cat_emb = embedding.embed_query("cute")

vectorstore.similarity_search_by_vector(cat_emb, k=2)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [17]:
await vectorstore.asimilarity_search("fish", k=2)

[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

# 5 - Retriever <div id="5"></div>

LangChain `VectorStore` objects do not subclass `Runnable`, and so cannot immediately be integrated into LangChain Expression Language chains.

We can create a simple version of runnable `VectorStore`, without subclassing Retriever. Below we will build one around the similarity_search method:

In [20]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) 
# .bind() pass parameter k=1 (select top result) whenever retriever is called

retriever.batch(["cat", "fish"])
# batch is using .invoke() in parallel using a thread pool executor

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

Fortunately, we can convert `VectorStore` into runnable retriever using `.as_retriever()` method, specifically become `VectorStoreRetriever`. These retrievers include specific `search_type` and `search_kwargs` attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [21]:
retriever = vectorstore.as_retriever(
    search_type="similarity", # alternatives: mmr, similarity_score_threshold
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "fish"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

# 6 - Create Chain with Retriever <div id="6"></div>

In [22]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

In [24]:
response = rag_chain.invoke("Tell be about cat")

print(response.content)

Cats are independent pets that often enjoy their own space.
